# Advent of Code 2020 - Day 2

In [7]:
data class Input(val password: String, val policy: Policy)

data class Policy(val range: IntRange, val string: String)

In [8]:
import java.io.File
val regex = Regex("""(\d*)-(\d*) (\w): (\w*)""")

val inputs: List<Input> = File("Day2.input.txt")
    .bufferedReader()
    .readLines()
    .map {
        regex
            .find(it)!!
            .destructured
            .let {(min, max, string, password) -> 
                Input(password = password, policy = Policy(range = min.toInt()..max.toInt(), string = string))
            }
    }

## Part 1

Find out how many passwords are valid according to their polices. A policy defines a `range` number of times a `string` can occur within the password. i.e. `Policy(range = 1..4, string = "a")` means the `password` must contain `a` at least `1` time and at most `4` times.

In [9]:
inputs
    .count { input ->
        input.password.windowed(size = input.policy.string.length, partialWindows = true)
            .count { it == input.policy.string } in input.policy.range
    }

414

### Notes

The idea is to break each password into "windows" of the size of the respective policy `string` and then simply count the number of times each window matches the respective policy `string`. If the count is between the min and max, it is a valid password.

Time complexity of `map` is `O(n)`, then the inner `windowed` and `fold` operations are `O(m)`, lastly the remaining `count` is `O(n)`. So the total time complexity is `O(n*m)`. This would be multi-linear, but I believe `m` is effectively a constant in this case, which would make it `O(n)` or linear-time.

Post-completion note: The solution to this problem could be simplified but I tried to predict the layering question by supporting variable length policy `string`... But I will keep the original implementation because I think it's more interesting. See simplified solution below:

In [10]:
inputs.count { input -> input.password.count { it == input.policy.string[0] } in input.policy.range }

414

## Part 2

Find out how many passwords are valid according to their policies. A policy defines a `range` which describes the position in the `password` that must contain the policy `string`. The password is valid when either LHS `range` or RHS `range` contains the policy `string`, but not both. i.e. `Policy(range = 1..4, string = "a")` means the `password` must contain `a` in either the zeroth or third index.

Note: I would normally rename `range` to something more appropriate, but I am lazy, so I will reuse my `inputs`. I would also make `string` a `Char` since we ended up not needing a multi-length string.

In [11]:
inputs.count { 
    (it.policy.range.first - 1 >= 0 && it.password[it.policy.range.first - 1] == it.policy.string[0]) xor 
        (it.policy.range.last - 1 >= 0 && it.password[it.policy.range.last - 1] == it.policy.string[0]) 
}

413

### Notes

Problem is basically a "how does your language write XOR". In Kotlin Booleans have an infix operator `xor`.